In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
import timeit



## Load Datasets

Load CIFAR10 dataset.


In [2]:
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

NUM_TRAIN = 19000
NUM_VAL = 1000

cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=T.ToTensor())
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=T.ToTensor())
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True,
                          transform=T.ToTensor())
loader_test = DataLoader(cifar10_test, batch_size=64)


Files already downloaded and verified


Files already downloaded and verified


Files already downloaded and verified


In [3]:
dtype = torch.FloatTensor  # the CPU datatype

# Constant to control how grequentl we print train loss
print_every = 100

# This is a little utility that we'll use to reset the model if we want to
# re-initialize all our parameters
def reset(m):
  if hasattr(m, "reset_parameters"):
    m.reset_parameters()

# define new module which only purpouse is to change dimension of input from
# convolusional convinient form into fully-connected convinient
class Flatten(nn.Module):
  def forward(self, x):
    N, C, H, W = x.size()  # read in dims sizes
    # 'flatten' the C * H * W values into a single vector pre image
    return x.view(N, -1)


## Definition of ConvNet model

In [4]:
# Here's where we define the architecture of the model
#          Conv2d       Flatten   Linear
# (32x32x3) -> (13x13x32) -> (5408) -> 10
simple_model = nn.Sequential(
             nn.Conv2d(3, 32, kernel_size=7, stride=2),
             nn.ReLU(inplace=True),
             Flatten(),
             nn.Linear(5408, 10),
             )

# Set the type of all data in this model to be FloatTensor
simple_model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)

# lr sets the learning rate of the optymizer
optymizer = optim.Adam(simple_model.parameters(), lr=1e-2)



More fancyier model

In [10]:
# Here's another funny model
#          Conv2d       MaxPool   Flatten     Linear    Linear
# (32x32x3) -> (26x26x32) -> (13x13x32) -> (5408) -> (1024) -> (10)
fixed_model_base = nn.Sequential(
                 nn.Conv2d(3, 32, kernel_size=7, stride=1),
                 nn.ReLU(inplace=True),
                 nn.BatchNorm2d(32, affine=False),
                 nn.MaxPool2d(kernel_size=2, stride=2),
                 Flatten(),
                 nn.Linear(5408, 1024),
                 nn.ReLU(inplace=True),
                 nn.Linear(1024, 10),
                 )

# Set the type of all data to be FloarTensor(CPU friendly)
fixed_model = fixed_model_base.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)

optymizer = optim.RMSprop(fixed_model.parameters(), lr=1e-2)


Sanity check - feed random batch into the model defined above and make sure the output is the right size

In [17]:
# Sanity check - feed random batch into the model defined above and make sure
# the output is the right size
x = torch.randn(64, 3, 32, 32).type(dtype)
x_var =  Variable(x)  # Construct a PyTorch Variable out of input data
ans = fixed_model(x_var)  # Feed it through the model

# Check to make sure what comes out of model
print("The output shape is: ", np.array(ans.size()))
print("Is valid? ", np.array_equal(np.array(ans.size()), np.array([64, 10])))



The output shape is:  [64 10]
Is valid?  True


Time single forward pass just for curiousity

In [18]:
%%timeit
ans = fixed_model(x_var)

163 ms ± 5.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Train model

In [ ]:
# Set model to 'training' mode. This is relevant for soe layers that may have
# different behavior in training mode vs testing mode, such as Dropout and BatchNorm
fixed_model.train()

# Load one batch at time
for t, (x, y) in enumerate(loader_train):
  x_var = Variable(x.type(dtype))
  y_var = Variable(y.type(dtype).long())

  # This is a forward pass: predict the scores for each class, for each x in the batch
  scores = fixed_model(x_var)

  # Use the correct y values and the  predicted y values to compute the loss.
  loss = loss_fn(scores, y_var)

  if (t + 1) % print_every == 0:
    print('t = {0}, loss={

